# Evaluate CBCC

In [ ]:
import sys

In [ ]:
!@cd

In [ ]:
import polars as pl
import pandas as pd
import numpy as np

np.random.seed(777)

In [ ]:
sys.path.append(f"../../main_experiment")
sys.path.append(f"../../methods")

In [ ]:
from cbcc_wrapper import CBCC

In [ ]:
from io_utils import get_accuracy, get_recall, load_dataset_profile, load_gt, load_human_responses

In [ ]:
human_df = pd.read_csv("../human_responses_with_gt.csv")
gt = human_df.filter(["task","gt"]).drop_duplicates(keep='last')
biased_tasks = gt[gt["gt"]==4]["task"].unique()  
gt = gt.set_index("task")
human_df = human_df.drop(["gt"], axis=1)

In [ ]:
df0 = human_df.copy()
df10 = human_df.copy()
df20 = human_df.copy()

In [ ]:
sp=[5,10,0]

In [ ]:
for i in range(sp[0]):
    aidf = pd.read_csv(f"../ai_responses/AI_{i}.csv")
    df10 = pd.concat([df10, aidf], axis=0, ignore_index=True)
for i in range(sp[1]):
    aidf = pd.read_csv(f"../ai_responses/AI_{i}.csv")
    df20 = pd.concat([df20, aidf], axis=0, ignore_index=True)

In [ ]:
def generate_methods(n_iter=100000, random_state=0):
    labels = [0,1,2,3,4,5]
    return {
        "CBCC_M=2" :  CBCC(labels, C=2),
        "CBCC_M=4" :  CBCC(labels, C=4),
        "CBCC_M=8" :  CBCC(labels, C=8),
    }

In [ ]:
def get_record(num_ai, iter, method, acc, biased_acc):
    return {
        "num_ai": num_ai,
        "iteration": iter,
        "method": method,
        "accuracy": acc,
        "recall": biased_acc,
    }

records = []
for i,df in enumerate([df10, df20,df0]):
    print(f"=== {sp[i]} AI Workers ===")
    for iter in range(5):
        methods = generate_methods(random_state=iter)
        for name, method in methods.items():
            print(f"--- {name} ---")
            ret = method.fit_predict(df, seed=iter*777)
            acc = get_accuracy(gt, ret)
            biased_acc = get_recall(gt, ret, biased_tasks)
            print(f"Accuracy: {acc}, Recall: {biased_acc}")
            record = get_record(sp[i], iter, name, acc, biased_acc)
            records.append(record)
result_df = pd.DataFrame(records)

In [ ]:
result_df.to_csv("cbcc_results.csv", index=False)